<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# FEC - Enrichissement du modèle de données

**Tags:** #fec #datamodel #finance #snippet #operation

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** Ce notebook permet d'enrichir la base de donnée "état financier".

## Input

### Import libraries

In [17]:
import re
import naas_data_product

### Setup Variables

In [18]:
# Inputs
input_folder_path = "/home/ftp/FEC-engine/outputs/FEC/BDD_FEC"

# Outputs
output_folder_path = "/home/ftp/FEC-engine/outputs/FEC/BDD_ENR"

## Model

### Récupération du dernier fichier input

In [19]:
df_input = get_last_df(input_folder_path)
print("✅ Row fetched:", len(df_input))
df_input.head(1)

📁 Last file: /home/ftp/FEC-engine/outputs/FEC/BDD_FEC/20230524164327_FEC_BDD_FEC.csv
✅ Row fetched: 4532


,NOM_FICHIER,DATE,COMPTE_NUM,RUBRIQUE_N3,RUBRIQUE_N4,DEBIT,CREDIT
0,000000000FEC20181231.txt,2018-01-01,20500000,"BREVETS, LICENCES, LOGICIELS..",A Nouveau,4006.6,0.0


### Enrichissement de la base

In [20]:
db_enr = df_input.copy()

# Ajout colonnes entité et période
db_enr["ENTITY"] = db_enr["NOM_FICHIER"].str[:9]
db_enr["PERIOD"] = db_enr["NOM_FICHIER"].str[12:-6]
db_enr["PERIOD"] = pd.to_datetime(db_enr["PERIOD"], format="%Y%m")
db_enr["PERIOD"] = db_enr["PERIOD"].dt.strftime("%Y-%m")

# Ajout colonne month et month_index
db_enr["DATE"] = pd.to_datetime(db_enr["DATE"])
db_enr["MONTH"] = db_enr["DATE"].dt.strftime("%b")
db_enr["MONTH_INDEX"] = db_enr["DATE"].dt.month

# Calcul de la valeur debit-crédit
db_enr["VALUE"] = (db_enr["DEBIT"]) - (db_enr["CREDIT"])

db_enr.head(5)

,NOM_FICHIER,DATE,COMPTE_NUM,RUBRIQUE_N3,RUBRIQUE_N4,DEBIT,CREDIT,ENTITY,PERIOD,MONTH,MONTH_INDEX,VALUE
0,000000000FEC20181231.txt,2018-01-01,20500000,"BREVETS, LICENCES, LOGICIELS..",A Nouveau,4006.60,0.0,000000000,2018-12,Jan,1,4006.60
1,000000000FEC20181231.txt,2018-01-01,20700000,FONDS COMMERCIAL,A Nouveau,68415.00,0.0,000000000,2018-12,Jan,1,68415.00
2,000000000FEC20181231.txt,2018-01-01,21540000,MATÉRIEL INDUSTRIEL,A Nouveau,49815.13,0.0,000000000,2018-12,Jan,1,49815.13
3,000000000FEC20181231.txt,2018-01-01,21810000,AMÉNAG. AGENCEMENTS LOCAUX,A Nouveau,4767.25,0.0,000000000,2018-12,Jan,1,4767.25
4,000000000FEC20181231.txt,2018-01-01,21830000,MATÉRIEL BUREAU ET INFORMATIQU,A Nouveau,5525.66,0.0,000000000,2018-12,Jan,1,5525.66


In [21]:
# Calcul résultat pour équilibrage bilan dans capitaux propre
db_rn = db_enr.copy()

db_rn = db_rn[db_rn["COMPTE_NUM"].str.contains(r"^6|^7")]

to_group = ["ENTITY", "PERIOD"]
to_agg = {"VALUE": "sum"}
db_rn = db_rn.groupby(to_group, as_index=False).agg(to_agg)

db_rn["COMPTE_NUM"] = "10999999"
db_rn["RUBRIQUE_N3"] = "RESULTAT"

# Reorganisation colonne
to_select = ["ENTITY", "PERIOD", "COMPTE_NUM", "RUBRIQUE_N3", "VALUE"]
db_rn = db_rn[to_select]
db_rn

,ENTITY,PERIOD,COMPTE_NUM,RUBRIQUE_N3,VALUE
0,000000000,2017-12,10999999,RESULTAT,-81496.79
1,000000000,2018-12,10999999,RESULTAT,-32807.85


### Base de données FEC aggrégée avec variation

#### Aggrégation RUBRIQUE N3

In [22]:
# Calcul var v = création de dataset avec Period_comp pour merge
db_var = db_enr.copy()

# Regroupement
to_group = ["ENTITY", "PERIOD", "COMPTE_NUM", "RUBRIQUE_N3"]
to_agg = {"VALUE": "sum"}
db_var = db_var.groupby(to_group, as_index=False).agg(to_agg)

# Ajout des résultats au dataframe
db_var = pd.concat([db_var, db_rn], axis=0, sort=False)

# Creation colonne COMP
db_var["PERIOD_COMP"] = (db_var["PERIOD"].str[:4].astype(int) - 1).astype(str) + db_var[
    "PERIOD"
].str[-3:]

db_var

,ENTITY,PERIOD,COMPTE_NUM,RUBRIQUE_N3,VALUE,PERIOD_COMP
0,000000000,2017-12,10130000,CAPITAL SOUSCRIT APPELÉ VERSÉ,-6000.00,2016-12
1,000000000,2017-12,10611000,RESERVE LEGALE,-600.00,2016-12
2,000000000,2017-12,10688000,AUTRES RESERVES,-130673.92,2016-12
3,000000000,2017-12,12000000,RÉSULTAT DE L'EXERCICE BÉNÉFIC,0.00,2016-12
4,000000000,2017-12,20500000,"BREVETS, LICENCES, LOGICIELS..",4006.60,2016-12
...,...,...,...,...,...,...
291,000000000,2018-12,77180000,AUTRES PROD EXCEP S/OP GESTION,-51.00,2017-12
292,000000000,2018-12,79100000,TRANSFERT DE CHARGES D'EXPLOIT,-46.07,2017-12
293,000000000,2018-12,79101000,RÉINT. UTILIS. PERSONNELLE,-1750.50,2017-12
0,000000000,2017-12,10999999,RESULTAT,-81496.79,2016-12


#### Création de la base comparable

In [23]:
db_comp = db_var.copy()

# Suppression de la colonne période
db_comp = db_comp.drop("PERIOD_COMP", axis=1)

# Renommage des colonnes
to_rename = {"VALUE": "VALUE_N-1", "PERIOD": "PERIOD_COMP"}
db_comp = db_comp.rename(columns=to_rename)

db_comp.head(5)

,ENTITY,PERIOD_COMP,COMPTE_NUM,RUBRIQUE_N3,VALUE_N-1
0,000000000,2017-12,10130000,CAPITAL SOUSCRIT APPELÉ VERSÉ,-6000.00
1,000000000,2017-12,10611000,RESERVE LEGALE,-600.00
2,000000000,2017-12,10688000,AUTRES RESERVES,-130673.92
3,000000000,2017-12,12000000,RÉSULTAT DE L'EXERCICE BÉNÉFIC,0.00
4,000000000,2017-12,20500000,"BREVETS, LICENCES, LOGICIELS..",4006.60


#### Jointure des 2 tables et calcul des variations

In [24]:
# Jointure entre les 2 tables
join_on = ["ENTITY", "PERIOD_COMP", "COMPTE_NUM", "RUBRIQUE_N3"]
db_var = (
    pd.merge(db_var, db_comp, how="left", on=join_on)
    .drop("PERIOD_COMP", axis=1)
    .fillna(0)
)

# Création colonne Var V
db_var["VARV"] = db_var["VALUE"] - db_var["VALUE_N-1"]

# Création colonne Var P (%)
db_var["VARP"] = db_var["VARV"] / db_var["VALUE_N-1"]

db_var

,ENTITY,PERIOD,COMPTE_NUM,RUBRIQUE_N3,VALUE,VALUE_N-1,VARV,VARP
0,000000000,2017-12,10130000,CAPITAL SOUSCRIT APPELÉ VERSÉ,-6000.00,0.00,-6000.00,-inf
1,000000000,2017-12,10611000,RESERVE LEGALE,-600.00,0.00,-600.00,-inf
2,000000000,2017-12,10688000,AUTRES RESERVES,-130673.92,0.00,-130673.92,-inf
3,000000000,2017-12,12000000,RÉSULTAT DE L'EXERCICE BÉNÉFIC,0.00,0.00,0.00,NaN
4,000000000,2017-12,20500000,"BREVETS, LICENCES, LOGICIELS..",4006.60,0.00,4006.60,inf
...,...,...,...,...,...,...,...,...
291,000000000,2018-12,77180000,AUTRES PROD EXCEP S/OP GESTION,-51.00,0.00,-51.00,-inf
292,000000000,2018-12,79100000,TRANSFERT DE CHARGES D'EXPLOIT,-46.07,0.00,-46.07,-inf
293,000000000,2018-12,79101000,RÉINT. UTILIS. PERSONNELLE,-1750.50,-1823.44,72.94,-0.040001
294,000000000,2017-12,10999999,RESULTAT,-81496.79,0.00,-81496.79,-inf


In [25]:
db_cat = db_var.copy()

# Calcul des rubriques niveau 2
def rubrique_N2(row):
    numero_compte = str(row["COMPTE_NUM"])
    value = float(row["VALUE"])

    # BILAN SIMPLIFIE type IFRS NIV2

    to_check = ["^10", "^11", "^12", "^13", "^14"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "CAPITAUX_PROPRES"

    to_check = ["^15", "^16", "^17", "^18", "^19"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "DETTES_FINANCIERES"

    to_check = ["^20", "^21", "^22", "^23", "^25", "^26", "^27", "^28", "^29"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "IMMOBILISATIONS"

    to_check = ["^31", "^32", "^33", "^34", "^35", "^36", "^37", "^38", "^39"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "STOCKS"

    to_check = ["^40"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "DETTES_FOURNISSEURS"

    to_check = ["^41"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "CREANCES_CLIENTS"

    to_check = ["^42", "^43", "^44", "^45", "^46", "^47", "^48", "^49"]
    if any(re.search(x, numero_compte) for x in to_check):
        if value > 0:
            return "AUTRES_CREANCES"
        else:
            return "AUTRES_DETTES"

    to_check = ["^50", "^51", "^52", "^53", "^54", "^58", "^59"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "DISPONIBILITES"

    # COMPTE DE RESULTAT DETAILLE NIV2

    to_check = ["^60"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "ACHATS"

    to_check = ["^61", "^62"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "SERVICES_EXTERIEURS"

    to_check = ["^63"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "TAXES"

    to_check = ["^64"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "CHARGES_PERSONNEL"

    to_check = ["^65"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "AUTRES_CHARGES"

    to_check = ["^66"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "CHARGES_FINANCIERES"

    to_check = ["^67"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "CHARGES_EXCEPTIONNELLES"

    to_check = ["^68", "^78"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "AMORTISSEMENTS"

    to_check = ["^69"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "IMPOT"

    to_check = ["^70"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "VENTES"

    to_check = ["^71", "^72"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "PRODUCTION_STOCKEE_IMMOBILISEE"

    to_check = ["^74"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "SUBVENTIONS_D'EXPL."

    to_check = ["^75", "^791"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "AUTRES_PRODUITS_GESTION_COURANTE"

    to_check = ["^76", "^796"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "PRODUITS_FINANCIERS"

    to_check = ["^77", "^797"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "PRODUITS_EXCEPTIONNELS"

    to_check = ["^78"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "REPRISES_AMORT._DEP."

    to_check = ["^8"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "COMPTES_SPECIAUX"


# Calcul des rubriques niveau 1
def rubrique_N1(row):
    categorisation = row.RUBRIQUE_N2

    # BILAN SIMPLIFIE type IFRS N1

    to_check = ["CAPITAUX_PROPRES", "DETTES_FINANCIERES"]
    if any(re.search(x, categorisation) for x in to_check):
        return "PASSIF_NON_COURANT"

    to_check = ["IMMOBILISATIONS"]
    if any(re.search(x, categorisation) for x in to_check):
        return "ACTIF_NON_COURANT"

    to_check = ["STOCKS", "CREANCES_CLIENTS", "AUTRES_CREANCES"]
    if any(re.search(x, categorisation) for x in to_check):
        return "ACTIF_COURANT"

    to_check = ["DETTES_FOURNISSEURS", "AUTRES_DETTES"]
    if any(re.search(x, categorisation) for x in to_check):
        return "PASSIF_COURANT"

    to_check = ["DISPONIBILITES"]
    if any(re.search(x, categorisation) for x in to_check):
        return "DISPONIBILITES"

    # COMPTE DE RESULTAT SIMPLIFIE N1

    to_check = ["ACHATS"]
    if any(re.search(x, categorisation) for x in to_check):
        return "COUTS_DIRECTS"

    to_check = [
        "SERVICES_EXTERIEURS",
        "TAXES",
        "CHARGES_PERSONNEL",
        "AUTRES_CHARGES",
        "AMORTISSEMENTS",
    ]
    if any(re.search(x, categorisation) for x in to_check):
        return "CHARGES_EXPLOITATION"

    to_check = ["CHARGES_FINANCIERES"]
    if any(re.search(x, categorisation) for x in to_check):
        return "CHARGES_FINANCIERES"

    to_check = ["CHARGES_EXCEPTIONNELLES", "IMPOT"]
    if any(re.search(x, categorisation) for x in to_check):
        return "CHARGES_EXCEPTIONNELLES"

    to_check = ["VENTES", "PRODUCTION_STOCKEE_IMMOBILISEE"]
    if any(re.search(x, categorisation) for x in to_check):
        return "CHIFFRE_D'AFFAIRES"

    to_check = [
        "SUBVENTIONS_D'EXPL.",
        "AUTRES_PRODUITS_GESTION_COURANTE",
        "REPRISES_AMORT._DEP.",
    ]
    if any(re.search(x, categorisation) for x in to_check):
        return "PRODUITS_EXPLOITATION"

    to_check = ["PRODUITS_FINANCIERS"]
    if any(re.search(x, categorisation) for x in to_check):
        return "PRODUITS_FINANCIERS"

    to_check = ["PRODUITS_EXCEPTIONNELS"]
    if any(re.search(x, categorisation) for x in to_check):
        return "PRODUITS_EXCEPTIONNELS"


# Calcul des rubriques niveau 0
def rubrique_N0(row):
    masse = row.RUBRIQUE_N1

    to_check = ["ACTIF_NON_COURANT", "ACTIF_COURANT", "DISPONIBILITES"]
    if any(re.search(x, masse) for x in to_check):
        return "ACTIF"

    to_check = ["PASSIF_NON_COURANT", "PASSIF_COURANT"]
    if any(re.search(x, masse) for x in to_check):
        return "PASSIF"

    to_check = [
        "COUTS_DIRECTS",
        "CHARGES_EXPLOITATION",
        "CHARGES_FINANCIERES",
        "CHARGES_EXCEPTIONNELLES",
    ]
    if any(re.search(x, masse) for x in to_check):
        return "CHARGES"

    to_check = [
        "CHIFFRE_D'AFFAIRES",
        "PRODUITS_EXPLOITATION",
        "PRODUITS_FINANCIERS",
        "PRODUITS_EXCEPTIONNELS",
    ]
    if any(re.search(x, masse) for x in to_check):
        return "PRODUITS"


# Mapping des rubriques
db_cat["RUBRIQUE_N2"] = db_cat.apply(lambda row: rubrique_N2(row), axis=1)
db_cat["RUBRIQUE_N1"] = db_cat.apply(lambda row: rubrique_N1(row), axis=1)
db_cat["RUBRIQUE_N0"] = db_cat.apply(lambda row: rubrique_N0(row), axis=1)


# Reorganisation colonne
to_select = [
    "ENTITY",
    "PERIOD",
    "COMPTE_NUM",
    "RUBRIQUE_N0",
    "RUBRIQUE_N1",
    "RUBRIQUE_N2",
    "RUBRIQUE_N3",
    "VALUE",
    "VALUE_N-1",
    "VARV",
    "VARP",
]
db_cat = db_cat[to_select]
db_cat

,ENTITY,PERIOD,COMPTE_NUM,RUBRIQUE_N0,RUBRIQUE_N1,RUBRIQUE_N2,RUBRIQUE_N3,VALUE,VALUE_N-1,VARV,VARP
0,000000000,2017-12,10130000,PASSIF,PASSIF_NON_COURANT,CAPITAUX_PROPRES,CAPITAL SOUSCRIT APPELÉ VERSÉ,-6000.00,0.00,-6000.00,-inf
1,000000000,2017-12,10611000,PASSIF,PASSIF_NON_COURANT,CAPITAUX_PROPRES,RESERVE LEGALE,-600.00,0.00,-600.00,-inf
2,000000000,2017-12,10688000,PASSIF,PASSIF_NON_COURANT,CAPITAUX_PROPRES,AUTRES RESERVES,-130673.92,0.00,-130673.92,-inf
3,000000000,2017-12,12000000,PASSIF,PASSIF_NON_COURANT,CAPITAUX_PROPRES,RÉSULTAT DE L'EXERCICE BÉNÉFIC,0.00,0.00,0.00,NaN
4,000000000,2017-12,20500000,ACTIF,ACTIF_NON_COURANT,IMMOBILISATIONS,"BREVETS, LICENCES, LOGICIELS..",4006.60,0.00,4006.60,inf
...,...,...,...,...,...,...,...,...,...,...,...
291,000000000,2018-12,77180000,PRODUITS,PRODUITS_EXCEPTIONNELS,PRODUITS_EXCEPTIONNELS,AUTRES PROD EXCEP S/OP GESTION,-51.00,0.00,-51.00,-inf
292,000000000,2018-12,79100000,PRODUITS,PRODUITS_EXPLOITATION,AUTRES_PRODUITS_GESTION_COURANTE,TRANSFERT DE CHARGES D'EXPLOIT,-46.07,0.00,-46.07,-inf
293,000000000,2018-12,79101000,PRODUITS,PRODUITS_EXPLOITATION,AUTRES_PRODUITS_GESTION_COURANTE,RÉINT. UTILIS. PERSONNELLE,-1750.50,-1823.44,72.94,-0.040001
294,000000000,2017-12,10999999,PASSIF,PASSIF_NON_COURANT,CAPITAUX_PROPRES,RESULTAT,-81496.79,0.00,-81496.79,-inf


## Output

### Sauvegarde des fichiers en csv

In [26]:
save_df(db_cat, output_folder_path)

✅ DataFrame saved in: /home/ftp/FEC-engine/outputs/FEC/BDD_ENR/20230524164843_FEC_BDD_ENR.csv
